# [1] 데이터 로더

In [0]:
! curl 'https://raw.githubusercontent.com/sejongresearch/2020.Spring.AI/master/HW/cabbage_price.csv' -o 'cabbage_price.csv'

print('Done..\n')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 93044  100 93044    0     0   499k      0 --:--:-- --:--:-- --:--:--  499k
Done..



In [0]:
import torch
import torch.optim as optim
import numpy as np

xy = np.loadtxt('cabbage_price.csv', delimiter=',', dtype=np.float32, skiprows=1, usecols=range(1,6))
x_data=xy[:, 0:-1]
y_data=xy[:,[-1]]

print(x_data[:5])   # x= 배추 값에 영향을 미치는 정보 : (평균온도, 최저온도, 최대온량, 강우량) 
print(y_data[:5])   # y= 실제 배추값

[[ -4.9 -11.    0.9   0. ]
 [ -3.1  -5.5   5.5   0.8]
 [ -2.9  -6.9   1.4   0. ]
 [ -1.8  -5.1   2.2   5.9]
 [ -5.2  -8.7  -1.8   0.7]]
[[2123.]
 [2123.]
 [2123.]
 [2020.]
 [2060.]]


In [0]:
x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([2922, 4])
torch.Size([2922, 1])


# [2] 모델 학습

In [0]:
W = torch.zeros((4, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-3) #1e-5

nb_epochs = 100000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    # 원래 위의 계산 :  hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    # Matrix 연산을 이용한 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @    

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선 (W,b 갱신)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 10000 == 0:  #10000
         print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
         ))


# lr =1e-2 를 넣으니까 발산하였다. 
# 따라서 lr은 보다 작은 값으로 지정하고, 에폭을 높여서 많이 돌려야 한다.

# lr=1e-3, 에폭 10만 => Cost: 1825343.750000 에서 더이상 cost값이 감소하지 않는다.
# 

Epoch    0/100000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 12610017.000000
Epoch 10000/100000 hypothesis: tensor([2720.0974, 3560.7305, 2550.8103,  ..., 2828.1123, 2667.6877,
        2555.0181]) Cost: 1873649.250000
Epoch 20000/100000 hypothesis: tensor([2805.9751, 3405.7097, 2772.0862,  ..., 2952.1890, 2868.4570,
        2806.5437]) Cost: 1831589.250000
Epoch 30000/100000 hypothesis: tensor([2841.0696, 3292.7761, 2848.5129,  ..., 2998.5195, 2945.2839,
        2901.7283]) Cost: 1826169.375000
Epoch 40000/100000 hypothesis: tensor([2854.2693, 3245.3691, 2875.8667,  ..., 3015.5107, 2973.6611,
        2936.7729]) Cost: 1825453.125000
Epoch 50000/100000 hypothesis: tensor([2859.1252, 3227.4189, 2885.7754,  ..., 3021.7122, 2984.0388,
        2949.5762]) Cost: 1825358.375000
Epoch 60000/100000 hypothesis: tensor([2860.8916, 3220.8037, 2889.3704,  ..., 3023.9666, 2987.8164,
        2954.2363]) Cost: 1825346.375000
Epoch 70000/100000 hypothesis: tensor([2861.5173, 3218.4136, 28

# [3] 정답 예측

#### 다음과 같은 정보가 주어졌을 때 예상되는 배추 값은 얼마인가?
평균 온도:-2.7   최저 온도:-6.6    최고 온도:2.0   강수량:0.1

In [0]:
# 가설설정 = 예측하고자 하는 값 = 배추 값
# H(x)= (-2.7w1) + (-6.6w2) + (2.0*w3) + (0.1*w4) +b

print(W)      #w1, w2, w3, w4
print(b)

tensor([[-79.3520],
        [ 38.2656],
        [ 62.2070],
        [  2.3790]], requires_grad=True)
tensor([2837.8457], requires_grad=True)


In [0]:
# 배추 값은
(-2.7 * -79.3520) + (-6.6 * 38.2656) + (2.0* 62.2070) + (0.1* 2.3790) +2837.8457

2924.19504

In [0]:
x_test= torch.tensor([-2.7, -6.6, 2.0, 0.1])
y_test=x_test.matmul(W)+b
print(y_test)

tensor([2924.1953], grad_fn=<AddBackward0>)
